In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime


day_dir='/Users/xinwang/stock/data/day/'
week_dir='/Users/xinwang/stock/data/week/'
report_dir='/Users/xinwang/stock/data/report/'
debug=False

level_30F = '30F'
level_day = 'Day'
level_week = 'Week'
k_columns = ['date','open','high','low','close','volume','amount']
day_level_prefix='D_'
week_level_prefix='W_'


In [2]:
import datetime as dt


def tostr(float_number):
    return '%.2f' % float_number

def get_fix_stop_loss_percent():
    return 7


# 止损价格
def get_stop_loss_price(price):
    return price * (1-get_fix_stop_loss_percent()/100)

def buy_how_many(account_money,price):
#     单笔交易承担亏损3%
    risk_percent = 3
    risk_money = account_money*risk_percent/100
    
#     止损价格
    stop_loss_price = get_stop_loss_price(price)
    plan_shares = risk_money/(price-stop_loss_price)
    
    return int((plan_shares)/100)*100

def sell_how_many(account_shares):
    return account_shares

def out_of_range(date,start_datetime,end_datetime):
    year_s, mon_s, day_s = date.split('-')
    cur_datetime = datetime(int(year_s), int(mon_s), int(day_s))
    
    return cur_datetime<start_datetime or cur_datetime>end_datetime
    
    
def to_datetime(date_str):
    return datetime.strptime(date_str,'%Y-%m-%d') 


def to_pd_datetime(date_str):
    var_datetime = to_datetime(date_str)
    return pd.datetime(var_datetime.year,var_datetime.month,var_datetime.day)


def merge_day_k_lines_to_one_week_k(day_k_lines):
    the_first_day = day_k_lines[:1]
    the_last_day = day_k_lines[-1:]

    w_date = the_last_day['date'].values[0]
    w_open = the_first_day['open'].values[0]
    w_high = day_k_lines['high'].max()
    w_low = day_k_lines['low'].min()
    w_close = the_last_day['close'].values[0]
    w_volume = day_k_lines['volume'].sum()
    w_amount = day_k_lines['amount'].sum()

    return [w_date,w_open,w_high,w_low,w_close,w_volume,w_amount]


def get_day_k_in_that_week(cur_date,day_k):
    year = int(cur_date.split('-')[0])
    week = to_datetime(cur_date).isocalendar()[1]

    return day_k[(day_k['year']==year) & (day_k['week']==week)]


def build_week_k_advance(day_file_name):
    stock_data = pd.read_csv(day_dir+day_file_name,header=None, names=k_columns)

    period_type = 'W'
    stock_data['date'] = pd.to_datetime(stock_data['date'])

    stock_data.set_index('date',inplace=True)

    w_data = stock_data.resample(period_type).last()

    w_data['open'] = stock_data['open'].resample(period_type).first()
    w_data['high'] = stock_data['high'].resample(period_type).max()
    w_data['low'] = stock_data['low'].resample(period_type).min()
    w_data['volume'] = stock_data['volume'].resample(period_type).sum()
    w_data['amount'] = stock_data['amount'].resample(period_type).sum()

    w_data.reset_index(inplace=True)
    w_data['date'] = w_data['date'].apply(lambda x: x + dt.timedelta(days = -2))
    w_data['datetime'] = w_data['date']
    w_data['date'] = w_data['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

    w_data = w_data[w_data['open'].notnull()]

    return w_data

def get_day_K(file_name):
    file = day_dir+file_name
    if(os.path.exists(file)):
        day_k = pd.read_csv(file, header=None, names=k_columns)

        day_k['datetime'] = pd.to_datetime(day_k['date'])
        day_k['year'] = day_k['date'].apply(lambda x: int(x.split('-')[0]))
        day_k['week'] = day_k['date'].apply(lambda x: to_datetime(x).isocalendar()[1])    

        return day_k
    else:
        return pd.DataFrame()
        

def get_week_K(day_k_file_name):
    original_name = day_k_file_name.split('_')[1]
    print(original_name)
    
    week_file = week_dir+ 'W_'+ original_name
    
    if(os.path.exists(week_file)):
        week_k =  pd.read_csv(week_file)
    else:
        print('No Week K ' + day_k_file_name)
        week_k = build_week_k_file_by_day_K(day_k_file_name)
    
    week_k['datetime'] = pd.to_datetime(week_k['date'])
#     week_k['year'] = week_k['date'].apply(lambda x: int(x.year))
#     week_k['week'] = week_k['date'].apply(lambda x: int(x.week))    
    
    return week_k
        

def build_week_k_file_by_day_K(day_k_file_name):
    week_k_df = build_week_k_advance(day_k_file_name)
    
    original_name = day_k_file_name.split('_')[1]
    week_file = week_dir +'W_'+ original_name
    
    week_k_df.to_csv(week_file, index=0)
    
    return week_k_df
    


In [3]:
class Metric:
            
    def __init__(self, day_k, week_k, level, short, mid):
        self.day_k = day_k
        self.week_k = week_k
        self.level = level
        self.short = short
        self.mid = mid
        
        if(self.level==level_day):
            self.data = self.day_k
        elif(self.level==level_week):
            self.data = self.week_k
        
    def previous_series(self,cur_date,latest_n):
        cur_index = self.data[self.data['date']==cur_date].index.values[0]
        start_index = cur_index.astype(int)-latest_n+1

        latest_part = self.data[start_index:cur_index+1]
            
        return latest_part
        
    def ma(self,cur_date,periods):
        periods_part = self.previous_series(cur_date, periods)['close']
        return round(periods_part.mean(),2)
    
    def std(self,cur_date, periods):
        periods_part = self.previous_series(cur_date, periods)['close']
        return np.std(periods_part)


    # {参数 N: 2，250，20 }
    # BOLL:MA(CLOSE,M); UB:BOLL+2*STD(CLOSE,M); LB:BOLL-2*STD(CLOSE,M);
    def boll(self,cur_date):
        M=20
        price = self.data[self.data['date']==cur_date]['close'].values[0]

        boll = self.ma(cur_date, M)
        up_boll = boll + 2 * (self.std(cur_date,M))
        low_boll = boll - 2 * (self.std(cur_date,M))
        return boll


    def is_on_boll(self,cur_date):
        boll_value = self.boll(cur_date)

        price = self.data[self.data['date']==cur_date]['close'].values[0]

        if(price>boll_value):
            return True
        else:
            return False
        
        
    def is_down_cross(self,cur_date):
            cur_ma_short = self.ma(cur_date,self.short)
            cur_ma_mid = self.ma(cur_date,self.mid)

            previous_part = self.previous_series(cur_date,2)
            if(len(previous_part)==0):
                return False

            last_day = previous_part['date'].values[0]

            last_day_ma_short = self.ma(last_day,self.short)
            last_day_ma_mid = self.ma(last_day,self.mid)

            if((cur_ma_short<cur_ma_mid) and (last_day_ma_short>=last_day_ma_mid)):
                return True
            else:
                return False
        
    def is_up_cross(self,cur_date):
        cur_ma_short = self.ma(cur_date,self.short)
        cur_ma_mid = self.ma(cur_date,self.mid)
  
        previous_part = self.previous_series(cur_date,2)
        if(len(previous_part)==0):
            return False
        
        last_day = previous_part['date'].values[0]
        
        last_day_ma_short = self.ma(last_day,self.short)
        last_day_ma_mid = self.ma(last_day,self.mid)
        
        if((cur_ma_short>cur_ma_mid) and (last_day_ma_short<=last_day_ma_mid)):
            return True
        else:
            return False

    def k_on_ma(self,cur_date, latest_n_k=3):
        latest_n_k_close = self.previous_series(cur_date,latest_n_k)['close']
        flag=True

        for c in latest_n_k_close:
            if(c<self.ma(cur_date,short)):
                return False

        return flag


    def latest_n_period_up_crossed(self,cur_date, latest_n=3):
        latest_part = self.previous_series(cur_date, latest_n)['date'].values

        cross_array = []
        for i in range(0,latest_n):
            date = latest_part[i]
            up_cross = self.is_up_cross(date, short, mid)

            if(up_cross == True):
                return True

        return False


    def latest_n_k_break_ma(self,cur_date, latest_n_k=3):
        latest_n_k_close = self.previous_series(cur_date,latest_n_k)['close']

        flag=True

        for c in latest_n_k_close:
            if(c>self.ma(cur_date, mid)):
                return False

        if(debug):
            print(cur_date + ' ' + str(flag))
        return flag
    
    def deep_k_break_ma(self,cur_date, deep_percent=3):
        price = self.data[self.data['date']==cur_date]['close'].values[0]
        ma_price = self.ma(cur_date, mid)
        if((ma_price-price)*100/price>deep_percent):
            return True

        return False

    def get_vol(self,cur_date):
        vol = self.data[self.data['date']==cur_date]['volume'].values[0]
        return vol

    def ma_vol(self,cur_date, periods=60):
        periods_part = self.previous_series(cur_date, periods)['volume']
        return periods_part.mean()

    def is_vol_outburst_point(self,cur_date, threshold=2, periods=60):
        vol = self.get_vol(cur_date)
        ma_vol_num = self.ma_vol(cur_date, periods)
        vol_rate = vol/ ma_vol_num

        if(vol_rate>threshold):
            return True
        else:
            return False

    # LARGE_VOL = SUM(VOL,5)/5/MA(VOL,60)    
    def is_latest_n_vol_outburst(self,cur_date, latest_n=5, threshold=2, periods=60):
        latest_part = self.previous_series(cur_date,latest_n)

        sum_vol = sum(latest_part['volume'])

        vol_rate = sum_vol/latest_n/self.ma_vol(cur_date, periods)

        if(vol_rate>threshold):
            return True
        else:
            return False


    # LARGE_VOL:=SUM(VOL,5)/5/MA(VOL,60)
    # COUNT(LARGE_VOL>2,10)>5
    # 成交量异动
    def is_outburst(self,cur_date, latest_n = 10, periods=60):
        latest_part = self.previous_series(cur_date,latest_n)

        count = 0
        for index,row in latest_part.iterrows():
            if(self.is_vol_outburst_point(row['date'])):
                count += 1

        if(count>5):
            return True
        else:
            return False

    def is_up_cross_outburst(self,cur_date, latest_n=10):
        latest_part = self.previous_series(cur_date, latest_n)

        for index,row in latest_part.iterrows():
            date = row['date']

            if(self.is_outburst(date)):
                return True

        next_part = self.next_periods_series(cur_date, latest_n)

        for index,row in next_part.iterrows():
            date = row['date']

            if(self.is_outburst(date)):
                return True

        return False
    


    def next_down_cross(self,next_sections):
        for index,row in next_sections.iterrows():
            date = row['date']
            if(self.is_down_cross(date)):
                return date

        return 'None'

    def previous_down_cross(self,next_sections):
        reverse_section = next_sections.iloc[::-1]

        for index,row in reverse_section.iterrows():
            date = row['date']
            if(self.is_down_cross(date)):
                return date

        return 'None'
        

In [4]:

class Section:
    
    def __init__(self, day_k, week_k, level, short, mid):
        self.day_k = day_k
        self.week_k = week_k
        self.level = level
        self.short = short
        self.mid = mid
        
        if(self.level==level_day):
            self.data = self.day_k
        elif(self.level==level_week):
            self.data = self.week_k
    
        self.metric = Metric(day_k, week_k, level, short, mid)
        
    
    def get_section(self,start_date_str,end_date_str):
        start = to_pd_datetime(start_date_str) 
        end = to_pd_datetime(end_date_str) 

        section = self.data[(self.data['datetime']>=start) & (self.data['datetime']<=end)]

        return section

    def get_highest_record(self,start_date,end_date):
        section = self.get_section(start_date,end_date)
        return section.loc[section['high'].idxmax()]

    def get_lowest_record(self,start_date,end_date):
        section = self.get_section(start_date,end_date)
        return section.loc[section['low'].idxmin()]

    def get_amplitude_percent(self,start_date,end_date):
        highest = self.get_highest_record(start_date,end_date)
        lowest = self.get_lowest_record(start_date,end_date)

        return round((highest['high']-lowest['low'])*100.0/lowest['low'],2)


    def how_many_periods(self,start_date,end_date):
        section = self.get_section(start_date,end_date)
        return section.shape[0]


    def get_ma_increase_per(self,start_date,end_date):        
        start_ma = self.metric.ma(start_date, self.short)
        end_ma = self.metric.ma(end_date, self.short)

        return round((end_ma-start_ma)*100.0/start_ma,2)


    

In [5]:
section_columns=['symbol','s_date','e_date','ampl','ma_per','periods','boll']

enable_day_boll_condition=True


class Report:
    section_df = pd.DataFrame(columns=section_columns)

    def __init__(self, file_name, level, short, mid):
        self.file_name = file_name
        self.day_k = get_day_K(file_name)
        self.week_k = get_week_K(file_name)
        
        self.level = level
        self.short = short
        self.mid = mid
        
        if(self.level==level_day):
            self.data = self.day_k
        elif(self.level==level_week):
            self.data = self.week_k
            
        
        self.start_date = self.data['date'][:1].values[0]
        self.end_date = self.data['date'][-1:].values[0]
        
        
        self.section_df = pd.DataFrame(columns=section_columns)
        self.section = Section(self.day_k, self.week_k, self.level, self.short, self.mid)
            
            
    def analyse_specific_day(self,cur_date):
        metric = Metric(self.day_k, self.week_k, self.level, self.short, self.mid)
        
        if(metric.is_up_cross(cur_date)):
            start = cur_date
            next_sections = self.section.get_section(start, self.end_date)
            previous_sections = self.section.get_section(self.start_date, start)

            next_down_cross_date = metric.next_down_cross(next_sections)
            
            if(next_down_cross_date == 'None'):
                return 
            
            if(debug==True):
                print('up cross:' +start + 
                      '\n down cross:' + next_down_cross_date)

            amplitude = self.section.get_amplitude_percent(start,next_down_cross_date)
            ma_percent = self.section.get_ma_increase_per(start,next_down_cross_date)
            periods = self.section.how_many_periods(start,next_down_cross_date)
            boll = metric.is_on_boll(cur_date)
            
            self.section_df.loc[self.section_df.shape[0] +1] = [self.file_name,start,next_down_cross_date,amplitude,ma_percent,periods,boll]

        
#     级别金叉，死叉，振幅，
    def analyse_up_cross_and_down_cross(self):
        all_dates = self.data['date'][self.mid:]
       
        start_datetime = to_datetime(self.start_date)
        end_datetime = to_datetime(self.end_date)

        progress = 0
        for date in all_dates:
            if(not(out_of_range(date,start_datetime,end_datetime))):
                self.analyse_specific_day(date)

            progress += 1
            if(progress % 500 == 0):
                print(tostr(progress*100/len(all_dates)) + '%', end=' ')
                
        return self.section_df
    
    
    
def run_gold_rate(symbols, level, short, mid):
    report_df = pd.DataFrame(columns=section_columns)
    
    for i in range(0, len(symbols)):
        report = Report(symbols[i], level,short,mid)
        temp_section = report.analyse_up_cross_and_down_cross()

        report_df = pd.concat([temp_section,report_df], ignore_index=True)
        
        for threshold in [60,70,80,90,100]:
            ampl_count = report_df[report_df['ampl']>threshold].shape[0]
            total = report_df.shape[0]
            if(total>0):
                rate = 100*ampl_count/total
                print('threshold '+ str(threshold) + ', ampl size:'+ str(ampl_count) +
                      ' total size:'+str(total) + ' rate ' +str(rate))
        print('------------------------- '+ str((i+1)*100.0/len(symbols)) +'% ---------------------------')
        
    now_str = datetime.now().strftime('%Y-%m-%d_%H:%M')
    report_df.to_csv(report_dir+level + '_' + now_str +'_report_df.csv',index=0)
    
    return report_df


symbols = ['D_SZ#000713.txt','D_SZ#002065.txt','D_SZ#002333.txt','D_SZ#002601.txt',
           'D_SZ#002882.txt','D_SZ#300207.txt','D_SZ#300476.txt',
           'D_SZ#000715.txt','D_SZ#002066.txt','D_SZ#002334.txt',
           'D_SZ#002602.txt','D_SZ#002883.txt','D_SZ#300208.txt','D_SZ#300477.txt',
           'D_SZ#000716.txt','D_SZ#002067.txt','D_SZ#002335.txt',
           'D_SZ#002603.txt','D_SZ#002884.txt','D_SZ#300209.txt','D_SZ#300478.txt',
           'D_SZ#000717.txt','D_SZ#002068.txt','D_SZ#002336.txt',
           'D_SZ#002604.txt','D_SZ#002885.txt','D_SZ#300210.txt','D_SZ#300479.txt',
           'D_SZ#000718.txt','D_SZ#002069.txt','D_SZ#002337.txt',
           'D_SZ#002605.txt','D_SZ#002886.txt','D_SZ#300211.txt','D_SZ#300480.txt',
           'D_SZ#300760.txt','D_SZ#000719.txt','D_SZ#002070.txt','D_SZ#002338.txt',
           'D_SZ#002606.txt','D_SZ#002887.txt','D_SZ#300212.txt','D_SZ#300481.txt',
           'D_SZ#000650.txt','D_SZ#002015.txt','D_SZ#002282.txt','D_SZ#002549.txt',
           'D_SZ#002825.txt','D_SZ#300159.txt','D_SZ#300428.txt','D_SZ#300701.txt',
           'D_SZ#000651.txt','D_SZ#002016.txt','D_SZ#002283.txt','D_SZ#002550.txt',
           'D_SZ#002826.txt','D_SZ#300160.txt','D_SZ#300429.txt','D_SZ#300702.txt',
           'D_SZ#000652.txt','D_SZ#002017.txt','D_SZ#002284.txt','D_SZ#002551.txt',
           'D_SZ#002827.txt','D_SZ#300161.txt','D_SZ#300430.txt','D_SZ#300703.txt',
           'D_SZ#000655.txt','D_SZ#002018.txt','D_SZ#002285.txt','D_SZ#002552.txt',
           'D_SZ#002828.txt','D_SZ#300162.txt','D_SZ#300431.txt','D_SZ#300705.txt',
           'D_SZ#000656.txt','D_SZ#002019.txt','D_SZ#002286.txt','D_SZ#002553.txt',
           'D_SZ#002829.txt','D_SZ#300163.txt','D_SZ#300432.txt','D_SZ#300706.txt',
           'D_SZ#000657.txt','D_SZ#002020.txt','D_SZ#002287.txt','D_SZ#002554.txt',
           'D_SZ#002830.txt','D_SZ#300164.txt','D_SZ#300433.txt','D_SZ#300707.txt',
           'D_SZ#000659.txt','D_SZ#002021.txt','D_SZ#002288.txt','D_SZ#002555.txt',
           'D_SZ#002831.txt','D_SZ#300165.txt','D_SZ#300434.txt','D_SZ#300708.txt',
           'D_SZ#000661.txt','D_SZ#002022.txt','D_SZ#002289.txt','D_SZ#002556.txt',
           'D_SZ#002832.txt','D_SZ#300166.txt','D_SZ#300435.txt','D_SZ#300709.txt',
           'D_SZ#000662.txt','D_SZ#002023.txt','D_SZ#002290.txt','D_SZ#002557.txt',
           'D_SZ#002833.txt','D_SZ#300167.txt','D_SZ#300436.txt','D_SZ#300710.txt',
           'D_SZ#000663.txt','D_SZ#002024.txt','D_SZ#002291.txt','D_SZ#002558.txt',
           'D_SZ#002835.txt','D_SZ#300168.txt','D_SZ#300437.txt','D_SZ#300711.txt']
# symbols = ['D_SZ#002022.txt']


day_report_df = run_gold_rate(symbols,level_day,short=30, mid=60)
day_report_df

week_report_df = run_gold_rate(symbols,level_week,short=30, mid=60)
week_report_df



def find_all_day_sections_between(symbol, start_date, end_date):
    start_datetime = to_datetime(start_date)
    end_datetime = to_datetime(end_date)
    
    day_report_df['s_datetime'] = pd.to_datetime(day_report_df['s_date'])
    
    part = day_report_df[(day_report_df['symbol']==symbol) & (day_report_df['s_datetime']>=start_datetime) & (day_report_df['s_datetime']<=end_datetime)]

    return part
    

def week_cross_and_day_cross():
    symbols = week_report_df['symbol'].drop_duplicates()
    
    ampl_df = pd.DataFrame(columns=['symbol','w_s_date','w_e_date','d_s_date','d_e_date','ampl','d_boll'])
    
    for symbol in symbols:
        dates = week_report_df[(week_report_df['symbol']==symbol)]['s_date']
        
        for date in dates:
            w_section = week_report_df[(week_report_df['symbol']==symbol) & (week_report_df['s_date']==date)]
            
            w_start_date = w_section['s_date'].values[0]
            w_end_date = w_section['e_date'].values[0]
            
            d_sections = find_all_day_sections_between(symbol, w_start_date, w_end_date)
                
            for index,row in d_sections.iterrows():
                d_s_date = row['s_date']
                d_e_date = row['e_date']
                d_ampl = row['ampl']
                d_boll = row['boll']
                
                ampl_df.loc[ampl_df.shape[0]+1] = [symbol,w_start_date, w_end_date,d_s_date,d_e_date,d_ampl,d_boll]
    
    now_str = datetime.now().strftime('%Y-%m-%d_%H:%M')
    ampl_df.to_csv(report_dir+'ampl_' + now_str +'_ampl_df.csv',index=0)
            
    return ampl_df
    
ampl_df = week_cross_and_day_cross()

print()
print('\n')
print('********************** The result ********************************')
        
for threshold in [60,70,80,90,100]:
    ampl_count = ampl_df[(ampl_df['ampl']>threshold) & (ampl_df['d_boll']==True)].shape[0]
    total = ampl_df.shape[0]
    rate = 100*ampl_count/total

    print('threshold '+ str(threshold) + ', ampl size:'+ str(ampl_count) +
          ' total size:'+str(total) + 
          ' rate ' +str(rate))
            
        
ampl_df


SZ#000713.txt
11.48% 22.96% 34.44% 45.92% 57.41% 68.89% 80.37% 91.85% threshold 60, ampl size:6 total size:38 rate 15.789473684210526
threshold 70, ampl size:4 total size:38 rate 10.526315789473685
threshold 80, ampl size:4 total size:38 rate 10.526315789473685
threshold 90, ampl size:4 total size:38 rate 10.526315789473685
threshold 100, ampl size:4 total size:38 rate 10.526315789473685
------------------------- 0.8130081300813008% ---------------------------
SZ#002065.txt
19.20% 38.40% 57.60% 76.80% 96.01% threshold 60, ampl size:13 total size:60 rate 21.666666666666668
threshold 70, ampl size:9 total size:60 rate 15.0
threshold 80, ampl size:9 total size:60 rate 15.0
threshold 90, ampl size:9 total size:60 rate 15.0
threshold 100, ampl size:9 total size:60 rate 15.0
------------------------- 1.6260162601626016% ---------------------------
SZ#002333.txt
23.63% 47.26% 70.89% 94.52% threshold 60, ampl size:15 total size:83 rate 18.072289156626507
threshold 70, ampl size:11 total size:8

67.20% threshold 60, ampl size:76 total size:366 rate 20.76502732240437
threshold 70, ampl size:63 total size:366 rate 17.21311475409836
threshold 80, ampl size:52 total size:366 rate 14.207650273224044
threshold 90, ampl size:47 total size:366 rate 12.841530054644808
threshold 100, ampl size:41 total size:366 rate 11.202185792349727
------------------------- 17.073170731707318% ---------------------------
SZ#000717.txt
11.39% 22.78% 34.18% 45.57% 56.96% 68.35% 79.74% 91.14% threshold 60, ampl size:85 total size:404 rate 21.03960396039604
threshold 70, ampl size:72 total size:404 rate 17.821782178217823
threshold 80, ampl size:58 total size:404 rate 14.356435643564357
threshold 90, ampl size:51 total size:404 rate 12.623762376237623
threshold 100, ampl size:45 total size:404 rate 11.138613861386139
------------------------- 17.88617886178862% ---------------------------
SZ#002068.txt
17.14% 34.28% 51.42% 68.56% 85.70% threshold 60, ampl size:91 total size:430 rate 21.162790697674417
th

/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars


36.13% 48.17% 60.21% 72.25% 84.30% 96.34% threshold 60, ampl size:108 total size:529 rate 20.415879017013232
threshold 70, ampl size:90 total size:529 rate 17.013232514177695
threshold 80, ampl size:72 total size:529 rate 13.610586011342155
threshold 90, ampl size:64 total size:529 rate 12.098298676748582
threshold 100, ampl size:58 total size:529 rate 10.964083175803403
------------------------- 23.577235772357724% ---------------------------
SZ#002069.txt
18.12% 36.23% 54.35% 72.46% 90.58% threshold 60, ampl size:114 total size:553 rate 20.61482820976492
threshold 70, ampl size:96 total size:553 rate 17.359855334538878
threshold 80, ampl size:77 total size:553 rate 13.924050632911392
threshold 90, ampl size:68 total size:553 rate 12.296564195298373
threshold 100, ampl size:62 total size:553 rate 11.211573236889693
------------------------- 24.390243902439025% ---------------------------
SZ#002337.txt
23.81% 47.62% 71.43% 95.24% threshold 60, ampl size:116 total size:574 rate 20.20905

threshold 60, ampl size:175 total size:831 rate 21.0589651022864
threshold 70, ampl size:145 total size:831 rate 17.448856799037305
threshold 80, ampl size:120 total size:831 rate 14.440433212996389
threshold 90, ampl size:105 total size:831 rate 12.635379061371841
threshold 100, ampl size:94 total size:831 rate 11.311672683513839
------------------------- 39.02439024390244% ---------------------------
SZ#300159.txt
28.51% 57.01% 85.52% threshold 60, ampl size:177 total size:848 rate 20.87264150943396
threshold 70, ampl size:146 total size:848 rate 17.21698113207547
threshold 80, ampl size:121 total size:848 rate 14.268867924528301
threshold 90, ampl size:106 total size:848 rate 12.5
threshold 100, ampl size:95 total size:848 rate 11.202830188679245
------------------------- 39.83739837398374% ---------------------------
SZ#300428.txt
68.97% threshold 60, ampl size:179 total size:856 rate 20.911214953271028
threshold 70, ampl size:147 total size:856 rate 17.172897196261683
threshold 80

...

59.67% threshold 60, ampl size:75 total size:112 rate 66.96428571428571
threshold 70, ampl size:68 total size:112 rate 60.714285714285715
threshold 80, ampl size:61 total size:112 rate 54.464285714285715
threshold 90, ampl size:60 total size:112 rate 53.57142857142857
threshold 100, ampl size:55 total size:112 rate 49.107142857142854
------------------------- 35.77235772357724% ---------------------------
SZ#002015.txt
86.96% threshold 60, ampl size:78 total size:116 rate 67.24137931034483
threshold 70, ampl size:71 total size:116 rate 61.206896551724135
threshold 80, ampl size:64 total size:116 rate 55.172413793103445
threshold 90, ampl size:61 total size:116 rate 52.58620689655172
threshold 100, ampl size:56 total size:116 rate 48.275862068965516
------------------------- 36.58536585365854% ---------------------------
SZ#002282.txt
threshold 60, ampl size:81 total size:119 rate 68.0672268907563
threshold 70, ampl size:74 total size:119 rate 62.18487394957983
threshold 80, ampl size:6

threshold 60, ampl size:113 total size:163 rate 69.32515337423312
threshold 70, ampl size:102 total size:163 rate 62.576687116564415
threshold 80, ampl size:93 total size:163 rate 57.05521472392638
threshold 90, ampl size:88 total size:163 rate 53.987730061349694
threshold 100, ampl size:82 total size:163 rate 50.306748466257666
------------------------- 52.84552845528455% ---------------------------
SZ#300430.txt
threshold 60, ampl size:113 total size:163 rate 69.32515337423312
threshold 70, ampl size:102 total size:163 rate 62.576687116564415
threshold 80, ampl size:93 total size:163 rate 57.05521472392638
threshold 90, ampl size:88 total size:163 rate 53.987730061349694
threshold 100, ampl size:82 total size:163 rate 50.306748466257666
------------------------- 53.65853658536585% ---------------------------
SZ#300703.txt
threshold 60, ampl size:113 total size:163 rate 69.32515337423312
threshold 70, ampl size:102 total size:163 rate 62.576687116564415
threshold 80, ampl size:93 tota

threshold 60, ampl size:149 total size:215 rate 69.30232558139535
threshold 70, ampl size:137 total size:215 rate 63.72093023255814
threshold 80, ampl size:125 total size:215 rate 58.13953488372093
threshold 90, ampl size:119 total size:215 rate 55.348837209302324
threshold 100, ampl size:112 total size:215 rate 52.093023255813954
------------------------- 69.91869918699187% ---------------------------
SZ#002554.txt
threshold 60, ampl size:152 total size:218 rate 69.72477064220183
threshold 70, ampl size:140 total size:218 rate 64.22018348623853
threshold 80, ampl size:127 total size:218 rate 58.25688073394495
threshold 90, ampl size:121 total size:218 rate 55.5045871559633
threshold 100, ampl size:114 total size:218 rate 52.293577981651374
------------------------- 70.73170731707317% ---------------------------
SZ#002830.txt
threshold 60, ampl size:152 total size:218 rate 69.72477064220183
threshold 70, ampl size:140 total size:218 rate 64.22018348623853
threshold 80, ampl size:127 to

threshold 60, ampl size:182 total size:258 rate 70.54263565891473
threshold 70, ampl size:167 total size:258 rate 64.72868217054264
threshold 80, ampl size:151 total size:258 rate 58.52713178294574
threshold 90, ampl size:145 total size:258 rate 56.201550387596896
threshold 100, ampl size:136 total size:258 rate 52.713178294573645
------------------------- 86.17886178861788% ---------------------------
SZ#300709.txt
threshold 60, ampl size:182 total size:258 rate 70.54263565891473
threshold 70, ampl size:167 total size:258 rate 64.72868217054264
threshold 80, ampl size:151 total size:258 rate 58.52713178294574
threshold 90, ampl size:145 total size:258 rate 56.201550387596896
threshold 100, ampl size:136 total size:258 rate 52.713178294573645
------------------------- 86.99186991869918% ---------------------------
SZ#000662.txt
60.10% threshold 60, ampl size:188 total size:268 rate 70.14925373134328
threshold 70, ampl size:172 total size:268 rate 64.17910447761194
threshold 80, ampl si

,symbol,w_s_date,w_e_date,d_s_date,d_e_date,ampl,d_boll
1,D_SZ#300168.txt,2012-05-11,2012-06-29,2012-06-08,2012-11-07,38.74,True
2,D_SZ#300168.txt,2012-08-24,2016-04-01,2013-01-09,2013-10-30,168.12,True
3,D_SZ#300168.txt,2012-08-24,2016-04-01,2014-02-11,2014-07-15,46.25,True
4,D_SZ#300168.txt,2012-08-24,2016-04-01,2014-08-28,2015-07-23,708.07,True
5,D_SZ#300168.txt,2012-08-24,2016-04-01,2016-02-04,2016-02-17,15.57,True
6,D_SZ#300168.txt,2012-08-24,2016-04-01,2016-02-25,2016-03-10,45.13,True
7,D_SZ#300168.txt,2012-08-24,2016-04-01,2016-03-18,2016-03-24,12.62,True
8,D_SZ#002558.txt,2013-05-24,2017-10-27,2013-06-14,2013-07-31,22.46,True
9,D_SZ#002558.txt,2013-05-24,2017-10-27,2013-10-11,2014-01-24,41.97,True
10,D_SZ#002558.txt,2013-05-24,2017-10-27,2014-07-07,2016-02-16,944.68,True


In [12]:
day_boll_df = day_report_df[day_report_df['boll']==True]

print(day_boll_df.shape[0])
for threshold in [60,70,80,90,100]:
    ampl_count = day_boll_df[(day_boll_df['ampl']>threshold)].shape[0]
    total = day_boll_df.shape[0]
    rate = 100*ampl_count/total

    print('threshold '+ str(threshold) + ', ampl size:'+ str(ampl_count) +
          ' total size:'+str(total) + 
          ' rate ' +str(rate))
            
        


1528
threshold 60, ampl size:389 total size:1528 rate 25.458115183246072
threshold 70, ampl size:315 total size:1528 rate 20.615183246073297
threshold 80, ampl size:262 total size:1528 rate 17.146596858638745
threshold 90, ampl size:224 total size:1528 rate 14.659685863874346
threshold 100, ampl size:197 total size:1528 rate 12.892670157068062


In [ ]:
def get_score_by_year_period_profits(year_period_profits):
    win_period = 0

    for i in range(0,len(year_period_profits)):
#         该年盈利百分比大于10%算为一个盈利年
        if(year_period_profits[i]>10):
            win_period += 1
            
#         该年盈利百分比大于10%算为一个盈利年
        if(year_period_profits[i]<-10):
            win_period -= 1
    
    return int(win_period*100/len(year_period_profits))
            
        
def get_score_by_max_retrace(retrace_percent):
    if(retrace_percent<=10):
        return 100-2*retrace_percent
    
    if(retrace_percent<20):
        return 80-2*(retrace_percent-10)
    
    if(retrace_percent<30):
        return 60-3*(retrace_percent-20)
    
    if(retrace_percent<40):
        return 30-3*(retrace_percent-30)
        
    if(retrace_percent<50):
        return 0-5*(retrace_percent-40)
    
    if(retrace_percent<60):
        return -50-5*(retrace_percent-50)
    
    return -100


In [ ]:
vol_columns = []
for i in range(0,20):
    vol_columns += ['v_f_' + str(i)]

section_columns = ['symbol','up_X','down_X','ampl','incre','periods','ma',
                   'pre_start','pre_end','outburst',
                   'pre_ampl','pre_incre','pre_periods',
                   'pre_ma'] + vol_columns
print( 'section_columns '+ str(len(section_columns))+'\n'+ str(section_columns))

class Eagle:
    file_name = ''
    start_date = ''
    end_date = ''
    short = 30
    mid = 60


    def __init__(self,file_name, short,mid):
        self.file_name = file_name
        self.short = short
        self.mid = mid
        
        self.data = get_day_level_k(file_name);
        
        self.start_date = self.data['date'][:1].values[0]
        self.end_date = self.data['date'][-1:].values[0]
        
        self.section_df = pd.DataFrame(columns=section_columns)
        
    
    def next_periods_series(self,cur_date, latest_n):
        print('next_periods_series ' + cur_date)
        cur_index = self.data[self.data['date']==cur_date].index.values[0]
        end_index = cur_index.astype(int)+latest_n

        latest_part = self.data[cur_index:end_index]
        return latest_part

    
    

    def is_buy_point(self,cur_date):
        latest_n=3
        up_crossed = self.latest_n_period_up_crossed(cur_date, latest_n)
        keep_up = self.k_on_ma(cur_date, self.short)

        if(up_crossed == True and keep_up == True):
            if(debug):
                print(cur_date+' up cross and keep 3 k lines up on MA('+ str(short)+')')
            return True
        else:
            return False

    def is_sell_point(self,cur_date):
        down_cross = self.is_down_cross(cur_date)

        latest_n = 3
        latest_n_break_ma = self.latest_n_k_break_ma(cur_date, latest_n)

        deep_percent=3
        deep_break_ma = self.deep_k_break_ma(cur_date,deep_percent)

        if(down_cross == True):
            if(debug):
                print('down_cross')
            return True

        if(latest_n_break_ma == True):
            if(debug):
                print('latest_'+str(latest_n)+'_break_ma_'+str(mid))
            return True

        if(deep_break_ma == True):
            if(debug):
                print('deep_'+str(deep_percent)+'%_break_ma_'+str(mid))
            return True

        return False
    
    
    def is_stop_loss(self,cur_date,last_buy_date,last_buy_price):
        price = self.data[self.data['date']==cur_date]['close'].values[0]

        loss_percent = get_fix_stop_loss_percent()
        stop_loss_price=last_buy_price*(1-loss_percent/100)
        if(price<stop_loss_price):
            return True
        else:
            return False


    # max_profit_rate % 0.6
    def is_stop_profit_v1(self,floating_profits,taking=40):
        if(len(floating_profits)<1):
            return False

        max_profit = max(floating_profits)
        last_one = floating_profits[-1]

        taking_from_max = max_profit*(1-taking/100)
        if(max_profit>10 and last_one<taking_from_max):
            return True
        else:
            return False


    def is_stop_profit(self,floating_profits,taking=40):
        return self.is_stop_profit_v1(floating_profits)



    def send_go_signal(self,outburst, pre_amplitude, pre_ma_per, amplitude_threshold=60):
        if((outburst==True) and (pre_ma_per>-10)):
            return 'Buy'

        return 'Hold'
    
    def get_vol_features(self,cur_date):
        vol = self.get_vol(cur_date)
        
        vol_features = []
        for long in [10,20,40,60]:
            vol_features.append(int(vol/self.ma_vol(cur_date,long)))
            for short in [2,4,8,16]:
                temp_feature = int(self.ma_vol(cur_date,short)/self.ma_vol(cur_date,long))
                vol_features.append(temp_feature)

        return vol_features

                

In [ ]:
class Account:
    init_money = 0
    start_date = ''
    account_money = 0
    account_shares = 0
    last_buy_price = 0
    last_buy_date = ''
    floating_profits = []
    operation_detail= pd.DataFrame(columns=['date','profit_rate','account_money'])
    daily_log=True
    
    def __init__(self,init_money,start_date,short,mid):
        self.init_money = init_money
        self.account_money = init_money
        self.start_date = start_date
        self.short = short
        self.mid = mid
        
        self.account_shares = 0
        self.last_buy_price = 0
        self.last_buy_date = ''
        self.floating_profits = []
        
        self.i = 0
        self.operation_detail = pd.DataFrame(columns=['date','profit_rate','account_money'])
        self.operation_detail.loc[self.i] = [start_date,0,self.init_money]
        self.i +=1
        
        self.daily_log=True
        
    
    def open_opsition(self,date,price):
        buy_shares = buy_how_many(self.account_money,price)
            
        if(self.daily_log==True):
            if(debug):
                print(date+' BUY price:'+ tostr(price) + ' shares:'+tostr(buy_shares)+ ' space:'+tostr(buy_shares*price*100/self.account_money))

        self.account_shares += buy_shares
        self.account_money -= buy_shares*price
        self.last_buy_price = price
        self.last_buy_date = date
        
    def sell_all(self,date, price, sell_type='sell_point'):
        sell_shares = sell_how_many(self.account_shares)
            
        self.account_shares -= sell_shares
        self.account_money += sell_shares*price
        
        profit_rate = (price-self.last_buy_price)*100/self.last_buy_price
        if(self.daily_log==True):
            if(debug):
                print(date +' '+sell_type+' ' +tostr(sell_shares)+' shares, profit rate:'+tostr(profit_rate)+'%')
                print('floating_profits:'+str(self.floating_profits))
                print()
            
        self.operation_detail.loc[self.i] = [date,profit_rate,self.account_money]
        self.i +=1
        self.floating_profits.clear()
            
    def daily_check(self,date,price):
        if(is_buy_point(date,self.short,self.mid)):
            if(self.account_shares==0):
                self.open_opsition(date,price)
                
        if(self.account_shares>0):
            floating_profit_percent = (price-self.last_buy_price)*100/self.last_buy_price
            self.floating_profits.append(round(floating_profit_percent,2))
            
#             if(is_stop_profit(self.floating_profits)):
#                 self.sell_all(date,price,'stop_profit')
            
            if(is_sell_point(date,self.short,self.mid)):
                self.sell_all(date,price,'sell_point')
                
            if(is_stop_loss(date,self.last_buy_date,self.last_buy_price)):
                self.sell_all(date,price,'stop_loss')
    
    def max_retrace(self):
        max_retrace_loss = 0
        
        moneys = self.operation_detail['account_money']
        for i in range(1,len(moneys)):
            history_part = moneys[:i+1]
            max_money = max(history_part)
            
            if(moneys[i]<max_money):
                retrace_percent = (max_money-moneys[i])*100/max_money
                
                if(retrace_percent>max_retrace_loss):
                    max_retrace_loss = retrace_percent

        return max_retrace_loss

    
    def continue_loss(self):
        max_loss_count = 0
        
        profits = self.operation_detail['profit_rate']
        for i in range(1,len(profits)):
            cur_profit = profits[i]
            
            if(cur_profit<=0):
                loss_count = 0
                j=i;
                while(j>=0 and profits[j]<=0):
                    loss_count+=1
                    j -= 1
                if(loss_count>max_loss_count):
                    max_loss_count = loss_count
                
        return max_loss_count
        
    def get_last_year_remain_money(self, year, the_first_year):
        if(year==the_first_year):
            return self.init_money
        
        last_year = year-1
        last_year_operations = self.operation_detail[self.operation_detail['year']==last_year]
        
        if(last_year_operations.shape[0]>0):
            return last_year_operations.loc[last_year_operations.shape[0]-1:]['account_money'].values[-1]
        else:
            return self.get_last_year_remain_money(last_year, the_first_year)
    
    def profit_per_year_period(self):
        self.operation_detail['year'] = self.operation_detail['date'].apply(lambda x: int(x.split('-')[0]))

        years_list = list(set(self.operation_detail['year'].values))
        
        years_list.sort()
        the_first_year = years_list[0]
        
        period_rates = []
        for i in range(0, len(years_list)):
            year = years_list[i]

            part = self.operation_detail[self.operation_detail['year']==year]
            remain_money = part.loc[part.shape[0]-1:]['account_money'].values[-1]
            
            last_year_remain_money = self.get_last_year_remain_money(year,the_first_year)
            
            growth_rate = (remain_money-last_year_remain_money)*100/last_year_remain_money
            period_rates.append(growth_rate)

        return period_rates
        
    
    def report(self):
        total_profit_per = (self.account_money-self.init_money)*100.0/self.init_money
        
        win_count = self.operation_detail[self.operation_detail['profit_rate']>0].shape[0]
        deal_count = self.operation_detail.shape[0]
        
        win_rate = win_count*100/deal_count
        continue_loss_times = self.continue_loss()
        max_retrace = self.max_retrace()
        
        period_rates = self.profit_per_year_period()
        period_profit_score = get_score_by_year_period_profits(period_rates)
        
        retrace_score = get_score_by_max_retrace(max_retrace)
        
        report_list = [self.short,self.mid,total_profit_per,
                       win_rate,continue_loss_times,max_retrace,
                       deal_count,win_count,
                       period_profit_score,retrace_score]
        
        return report_list
        

In [ ]:
report_columns = ['ma_short','ma_mid','total_profit',
                  'win_rate','loss_times','max_retrace',
                  'deal_count','win_count',
                  'period_score','retrace_score']

def buy_and_sell(start_date,end_date,report_summary, short,mid):
    print('start go through MA(' + str(short) + ',' + str(mid) + ')')
    all_dates = data['date'][mid:]
    
    account = Account(initial_money,start_date,short,mid)
    
    start_year_s, start_mon_s, start_day_s = start_date.split('-')
    start_datetime = datetime(int(start_year_s), int(start_mon_s), int(start_day_s))

    end_year_s, end_mon_s, end_day_s = end_date.split('-')
    end_datetime = datetime(int(end_year_s), int(end_mon_s), int(end_day_s))
    
    for date in all_dates:
        if(not(out_of_range(date,start_datetime,end_datetime))):
            price = data[data['date']==date]['close'].values[0]
            account.daily_check(date,price)
   
    report_data = account.report()
    report_summary.loc[report_summary.shape[0]+1] = report_data

    return report_summary

In [ ]:
def test():
    initial_money=100000

    report_summary = pd.DataFrame(columns=report_columns)
    summary = buy_and_sell('2007-07-05','2010-10-18',report_summary, short=10, mid=50)
    print(summary)
    print('Done')

    
# test()

In [ ]:
def grid_search_ma():
    shorts = range(5,40,10)
    mids = range(10,80,10)
    
    initial_money=100000
    report_summary = pd.DataFrame(columns=report_columns)
    
    for short_item in shorts:
        for mid_item in mids:
            if(mid_item>short_item):
                summary = buy_and_sell('2011-06-02','2019-02-15', report_summary, short_item, mid_item)
                print('done search MA(' + str(short_item) + ',' + str(mid_item) + ')')

    return report_summary

# summary = grid_search_ma()
# summary.sort_values(by=['total_profit','retrace_score','period_score'], ascending=False)